# Notebook 2 — Regresión Lineal con datos de accidentes y seguros

**Objetivo de la sesión**  
Entrenar un primer modelo de **regresión lineal** para predecir **primas de seguro de auto** usando información de accidentes, y entender:

- Cómo separar datos en **train / test**.  
- Cómo entrenar `LinearRegression` de `scikit-learn`.  
- Cómo evaluar el modelo con **MAE, RMSE, R²**.  
- Cómo analizar **residuales** y **coeficientes** (variables más influyentes).

Dataset (columnas utilizadas):

- `State`  
- `Number of drivers involved in fatal collisions per billion miles`  
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding`  
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired`  
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted`  
- `Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents`  
- `Car Insurance Premiums ($)`  
- `Losses incurred by insurance companies for collisions per insured driver ($)`


In [ ]:
# ==========================
# 0. Importar librerías
# ==========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

plt.style.use("default")
plt.rcParams["figure.figsize"] = (8, 5)


## 1. Cargar el dataset crudo

En esta celda cargamos el archivo CSV original **sin modificaciones previas**.


Ajusta la ruta al archivo según dónde lo tengas guardado (por ejemplo: `"bad-drivers.csv"` o `"data/bad-drivers.csv"`).


In [ ]:
# ==========================
# 1. Cargar datos crudos
# ==========================
DATA_PATH = "bad-drivers.csv"

df_raw = pd.read_csv(DATA_PATH)

# Ver primeras filas para confirmar estructura
df_raw.head()



## 2. Inspección general del dataset

Antes de modelar, necesitamos entender:

- Tipos de datos.  
- Valores faltantes.  
- Rangos de las variables numéricas.


In [ ]:
# Información general del dataset crudo
df_raw.info()


In [ ]:
# Estadísticas descriptivas de las variables numéricas
df_raw.describe().T


**Preguntas para comentar en grupo (rápido):**

1. ¿Hay valores faltantes en alguna columna relevante?  
2. ¿Alguna variable tiene un rango muy distinto a las demás (por ejemplo, más grande por varias órdenes de magnitud)?  
3. ¿Cuál podría ser una buena **variable objetivo** (y) para un modelo de regresión?

En este notebook asumiremos que queremos **predecir la prima de seguro**:

> `y = Car Insurance Premiums ($)`

## 3. Selección de variables para el modelo

Vamos a construir un modelo sencillo que prediga:
- `Car Insurance Premiums ($)` (variable objetivo)

Usando como **features**:

- `Number of drivers involved in fatal collisions per billion miles`
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding`
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired`
- `Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted`
- `Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents`
- `Losses incurred by insurance companies for collisions per insured driver ($)`

La columna `State` la dejaremos fuera del primer modelo (es categórica y requeriría codificación adicional).


In [ ]:
# ==========================
# 3. Seleccionar X (features) e y (target)
# ==========================

target_col = "Car Insurance Premiums ($)"

feature_cols = [
    "Number of drivers involved in fatal collisions per billion miles",
    "Percentage Of Drivers Involved In Fatal Collisions Who Were Speeding",
    "Percentage Of Drivers Involved In Fatal Collisions Who Were Alcohol-Impaired",
    "Percentage Of Drivers Involved In Fatal Collisions Who Were Not Distracted",
    "Percentage Of Drivers Involved In Fatal Collisions Who Had Not Been Involved In Any Previous Accidents",
    "Losses incurred by insurance companies for collisions per insured driver ($)"
]

# Nos quedamos solo con las columnas necesarias
df_model = df_raw[feature_cols + [target_col]].copy()

# Por simplicidad, eliminamos filas con NaN en estas columnas
df_model = df_model.dropna()

print(df_model.shape)
df_model.head()


## 4. Separar en entrenamiento y prueba (train / test split)

Usaremos:

- **X**: matriz de features.  
- **y**: vector de primas de seguro.  
- 80% de los datos para **entrenar** y 20% para **probar** el modelo.

Usaremos `random_state` fijo para reproducibilidad.


In [ ]:
# ==========================
# 4. Train / Test Split
# ==========================

X = df_model[feature_cols]
y = df_model[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape


**Chequeo rápido:**

- ¿Cuántos registros quedaron para entrenamiento y cuántos para prueba?  
- ¿Ves algún desbalance evidente (muchos pocos datos)?



---

## 5. Entrenar el modelo de Regresión Lineal

Usaremos `LinearRegression()` de `scikit-learn`:

- Ajusta un modelo lineal de la forma
  $\hat{y} = \beta_0 + \beta_1 x_1 + \dots + \beta_p x_p$


In [ ]:
# ==========================
# 5. Entrenar modelo LinearRegression
# ==========================

linreg = LinearRegression()
linreg.fit(X_train, y_train)

print("Intercepto (beta_0):", linreg.intercept_)
print("\nCoeficientes:")
for col, coef in zip(feature_cols, linreg.coef_):
    print(f"{col}: {coef:.4f}")


**Interpretación rápida de coeficientes:**

- El intercepto es el valor de la prima de seguro cuando todas las variables explicativas valen 0 (aquí tiene menos interpretación directa, pero es parte del modelo).  
- Cada coeficiente indica cómo cambia la **prima de seguro (en dólares)** cuando la variable correspondiente aumenta en 1 unidad, manteniendo las demás constantes.

---

## 6. Predicción y análisis de residuales

Ahora vamos a:

1. Obtener predicciones en el **conjunto de prueba**.  
2. Calcular **residuales**:  
   $ \text{residual} = y_{\text{real}} - y_{\text{predicho}}$ 
   
3. Graficar:
   - `y_test` vs `y_pred` (dispersión).  
   - Residuales vs `y_pred`.  
   - Histograma de residuales.


In [ ]:
# ==========================
# 6. Predicciones en el set de prueba
# ==========================

y_pred = linreg.predict(X_test)

# Creamos un DataFrame auxiliar para análisis
df_results = pd.DataFrame({
    "y_test": y_test.values,
    "y_pred": y_pred
})
df_results["residual"] = df_results["y_test"] - df_results["y_pred"]

df_results.head()


In [ ]:
# Dispersión: valores reales vs predichos
plt.figure()
plt.scatter(df_results["y_test"], df_results["y_pred"])
plt.xlabel("Prima real ($)")
plt.ylabel("Prima predicha ($)")
plt.title("Valores reales vs predichos")
plt.plot(
    [df_results["y_test"].min(), df_results["y_test"].max()],
    [df_results["y_test"].min(), df_results["y_test"].max()],
    linestyle="--"
)
plt.show()


In [ ]:
# Histograma de residuales
plt.figure()
plt.hist(df_results["residual"], bins=10)
plt.xlabel("Residual")
plt.ylabel("Frecuencia")
plt.title("Distribución de residuales")
plt.show()


**Preguntas de análisis (para comentario rápido):**

- ¿Los puntos en `y_real vs y_pred` se acercan a la diagonal?  
- ¿Los residuales parecen estar centrados alrededor de 0?  
- ¿Ves patrones claros (curvas, abanicos) en la gráfica de residuales vs predicción?  
  - Si hay patrones fuertes, podría indicar que el modelo lineal es insuficiente.

---

## 7. Métricas de desempeño: MAE, RMSE, R²

Vamos a calcular tres métricas clave:

- **MAE** (Mean Absolute Error): error medio absoluto.  
- **RMSE** (Root Mean Squared Error): error cuadrático medio en escala original.  
- **R²**: proporción de varianza explicada por el modelo (entre 0 y 1, aunque puede ser negativa si el modelo es muy malo).


In [ ]:
# ==========================
# 7. Cálculo de métricas
# ==========================

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²  : {r2:.3f}")


**Interpretación:**

- **MAE** y **RMSE** están en **dólares**, por lo que se pueden comparar directamente con la magnitud típica de la prima.  
- **R²** cerca de 1 indica que el modelo explica bien la variabilidad; cerca de 0 indica bajo poder explicativo.

Discute con tu grupo:
- Dado el rango de `Car Insurance Premiums ($)`, ¿estos errores son razonables?  
- ¿El R² parece aceptable para un primer modelo simple?

---

## 8. Variable más influyente (coeficientes)

Vamos a ordenar los coeficientes del modelo según su valor absoluto para ver qué variables tienen mayor impacto en la prima de seguro (bajo el supuesto lineal del modelo).


In [ ]:
# ==========================
# 8. Análisis de importancia de variables
# ==========================

coef_df = pd.DataFrame({
    "feature": feature_cols,
    "coef": linreg.coef_
})
coef_df["abs_coef"] = coef_df["coef"].abs()

coef_df_sorted = coef_df.sort_values("abs_coef", ascending=False)
coef_df_sorted


In [ ]:
# Gráfico de barras de la magnitud de los coeficientes
plt.figure()
plt.barh(coef_df_sorted["feature"], coef_df_sorted["coef"])
plt.xlabel("Coeficiente (impacto en la prima $)")
plt.title("Importancia de variables según regresión lineal")
plt.gca().invert_yaxis()  # la más importante arriba
plt.show()


## 8.1 Mejora rápida: Eliminación de Outliers por Residuos

Para asegurar una mejora en el modelo, vamos a identificar los puntos donde el modelo actual comete los errores más grandes (residuos altos) y los eliminaremos, ya que probablemente sean casos atípicos que ensucian la tendencia general.

In [141]:
# Calculamos los residuos del modelo original (diferencia entre real y predicho)

linreg_total = LinearRegression().fit(X, y)
residuos = np.abs(y - linreg_total.predict(X))

# Filtramos los datos que tengan un error menor a 1.5 desviaciones estándar
# Esto elimina los datos más problemáticos que se desvían de la tendencia
std_residuo = residuos.std()
df_clean = df_model[residuos < 1.5 * std_residuo]

# Entrenamos de nuevo con los datos limpios
X_train2, X_test2, y_train2, y_test2 = train_test_split(df_clean[feature_cols], df_clean[target_col], test_size=0.2, random_state=42)
linreg2 = LinearRegression().fit(X_train2, y_train2)

print(f"R² Original: {r2:.3f}")
print(f"R² Mejorado: {r2_score(y_test2, linreg2.predict(X_test2)):.3f}")
print(f"MSE Original: {mean_squared_error(y_test, linreg_total.predict(X_test)):.3f}")
print(f"MSE Mejorado: {mean_squared_error(y_test2, linreg2.predict(X_test2)):.3f}")
print(f"MAE Original: {mean_absolute_error(y_test, linreg_total.predict(X_test)):.3f}")
print(f"MAE Mejorado: {mean_absolute_error(y_test2, linreg2.predict(X_test2)):.3f}\n")
print(f"Datos eliminados: {len(df_model) - len(df_clean)}")

R² Original: -0.155
R² Mejorado: 0.603
MSE Original: 23641.706
MSE Mejorado: 6254.461
MAE Original: 119.546
MAE Mejorado: 66.588

Datos eliminados: 23


**Mini-desafío:**

En grupo, respondan:

1. ¿Cuál es la **variable más influyente** según la magnitud del coeficiente?  
2. ¿El signo del coeficiente (positivo/negativo) tiene sentido con la intuición del problema?  
3. Usando tanto la tabla de coeficientes como las gráficas de residuales:  
   - ¿Creen que este modelo lineal captura bien la relación entre variables?  
   - ¿Qué mejoraría para una siguiente versión (más features, transformaciones, modelos no lineales, etc.)?
4. ¿Consideran que vale la pena eliminar una gran cantidad de filas por un mejor modelo en un dominio acotado?
---

## 9. Cierre del Notebook 2

En este notebook aprendimos a:

- Partir desde el **dataset crudo**.  
- Seleccionar una **variable objetivo** y un subconjunto de **features**.  
- Dividir en **train/test**.  
- Entrenar un modelo de **regresión lineal** con `scikit-learn`.  
- Evaluar el desempeño con **MAE, RMSE, R²**.  
- Analizar **residuales** y **coeficientes** para entender el comportamiento del modelo.

En el siguiente paso, pueden:
- Probar diferentes combinaciones de features.  
- Agregar transformaciones (por ejemplo, escalar, log-transform).  
- Comparar con otros modelos de regresión más avanzados.

> Dejen guardado este notebook, porque será la base para conectarlo con otras sesiones del curso (por ejemplo, comparación con modelos más complejos o incorporación de nuevas variables).
